In [ ]:
import os
import pickle
# 수학 관련 라이브러리
import numpy as np
import math
# pytorch 관련 라이브러리
import torch
import torch.nn as nn 
import torch.nn.functional as F 

import pandas as pd
import nltk
import tqdm
from torch.utils import data # dataset 관련된 utility 를 사용하려는 용도
from random import choice, randrange # random
from itertools import zip_longest 
import json 
import random
import pdb


nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_files.zip to data_files.zip


In [ ]:
!unzip data_files.zip

Archive:  data_files.zip
  inflating: english_id2word.pkl     
  inflating: english_vocab.pkl       
  inflating: english_word2id.pkl     
   creating: korean_data/
  inflating: korean_data/test_english.csv  
  inflating: korean_data/test_korean.csv  
  inflating: korean_data/train.csv   
  inflating: korean_id2word.pkl      
  inflating: korean_vocab.pkl        
  inflating: korean_word2id.pkl      
  inflating: train_english.pkl       
  inflating: train_korean.pkl        


In [ ]:
def split_last(x, shape):
    "split the last dimension to given shape"
    shape = list(shape)
    assert shape.count(-1) <= 1
    if -1 in shape:
        shape[shape.index(-1)] = int(x.size(-1) / -np.prod(shape))
    return x.view(*x.size()[:-1], *shape)

def merge_last(x, n_dims):
    "merge the last n_dims to a dimension"
    s = x.size()
    assert n_dims > 1 and n_dims < len(s)
    return x.view(*s[:-n_dims], -1)

# Activation function

In [ ]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

# Layer normalization

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, cfg, variance_epsilon=1e-12):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(cfg.dim))
        self.beta  = nn.Parameter(torch.zeros(cfg.dim))
        self.variance_epsilon = variance_epsilon

    def forward(self, x):
        # get mean, variance
        u = x.mean(-1, keepdim=True) # sequence 방향 mean
        s = (x - u).pow(2).mean(-1, keepdim=True) # sequence 방향 variance
        
        # normalize
        x = (x - u) / torch.sqrt(s + self.variance_epsilon) # (x - mean)/std 
        
        return self.gamma * x + self.beta # gamma, beta를 이용해 mean, std 조정

# Embedding

In [ ]:
def get_sinusoid_encoding_table(n_position, d_model):
    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / d_model)
    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
    return torch.FloatTensor(sinusoid_table)


class Embeddings(nn.Module):
    "The embedding module from word, position and token_type embeddings."
    def __init__(self, cfg, vocab_size):
        super().__init__()
        
        self.tok_embed = nn.Embedding(vocab_size, cfg.dim) # token embedding
        self.pos_embed = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(512, cfg.dim),freeze=True) # position embedding

        self.norm = LayerNorm(cfg)
        self.drop = nn.Dropout(cfg.p_drop_hidden)

    def forward(self, x):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long, device=x.device) # 0,1,2,3,4,5, ..., seq_len-1
        pos = pos.unsqueeze(0).expand_as(x) # (S,) -> (B, S)

        e = self.tok_embed(x) + self.pos_embed(pos)
        return self.drop(self.norm(e))

#  Transformer encoder

In [ ]:
class Attention(nn.Module):
    #Scaled Dot Product Attention
    
    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1)) # scale
        
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn

In [ ]:
class MultiHeadAttention(nn.Module):
    """ Multi-Headed Dot Product Attention """
    def __init__(self, cfg):
        super().__init__()
        self.proj_q = nn.Linear(cfg.dim, cfg.dim)
        self.proj_k = nn.Linear(cfg.dim, cfg.dim)
        self.proj_v = nn.Linear(cfg.dim, cfg.dim)
        self.drop = nn.Dropout(cfg.p_drop_attn)
        self.scores = None # for visualization
        self.n_heads = cfg.n_heads

    def forward(self, x, mask, x_q=None):
        """
        x, q(query), k(key), v(value) : (B(batch_size), S(seq_len), D(dim))
        mask : (B(batch_size) x S(seq_len))
        * split D(dim) into (H(n_heads), W(width of head)) ; D = H * W
        """
        
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        if x_q is None:
            q, k, v = self.proj_q(x), self.proj_k(x), self.proj_v(x)
        else:
            q, k, v = self.proj_q(x_q), self.proj_k(x), self.proj_v(x)
        q, k, v = (split_last(x, (self.n_heads, -1)).transpose(1, 2)
                   for x in [q, k, v])
        # (B, H, S, W) @ (B, H, W, S) -> (B, H, S, S) -softmax-> (B, H, S, S)
        scores = torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(k.size(-1)) # @ == torch.matmul (dot product)
        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1)
            scores = scores.masked_fill_(mask, -1e9)
        scores = self.drop(F.softmax(scores, dim=-1))
        # (B, H, S, S) @ (B, H, S, W) -> (B, H, S, W) -trans-> (B, S, H, W)
        h = torch.matmul(scores, v).transpose(1,2).contiguous()
        # -merge-> (B, S, D)
        h = merge_last(h, 2)
        self.scores = scores
        return h

# Base feedforward network

In [ ]:
class PositionWiseFeedForward(nn.Module):
    """ FeedForward Neural Networks for each position """
    def __init__(self, cfg):
        super().__init__()
        self.fc1 = nn.Linear(cfg.dim, cfg.dim_ff)
        self.fc2 = nn.Linear(cfg.dim_ff, cfg.dim)

    def forward(self, x):
        # (B, S, D) -> (B, S, D_ff) -> (B, S, D)
        return self.fc2(gelu(self.fc1(x)))

# Transformer

In [ ]:
class Encoder_Block(nn.Module):
    """ Transformer Block """
    def __init__(self, cfg):
        super().__init__()
        self.attn = MultiHeadAttention(cfg)
        self.proj = nn.Linear(cfg.dim, cfg.dim)
        self.norm1 = LayerNorm(cfg)
        self.pwff = PositionWiseFeedForward(cfg)
        self.norm2 = LayerNorm(cfg)
        self.drop = nn.Dropout(cfg.p_drop_hidden)

    def forward(self, x, mask):
        h = self.attn(x, mask)
        h = self.norm1(x + self.drop(self.proj(h)))
        h = self.norm2(h + self.drop(self.pwff(h)))
        return h
    
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k
    
def get_attn_subsequent_mask(seq):
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequent_mask = np.triu(np.ones(attn_shape), k=1)
    subsequent_mask = torch.tensor(subsequent_mask, device=seq.device).byte()
    return subsequent_mask
    
    
class Decoder_Block(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.self_attention = MultiHeadAttention(cfg)
        self.encoder_attention = MultiHeadAttention(cfg)
        
        self.norm1 = LayerNorm(cfg)
        self.proj1 = nn.Linear(cfg.dim, cfg.dim)
        self.norm2 = LayerNorm(cfg)
        self.proj2 = nn.Linear(cfg.dim, cfg.dim)
        
        self.pwff = PositionWiseFeedForward(cfg)
        self.norm3 = LayerNorm(cfg)
        
        self.drop = nn.Dropout(cfg.p_drop_hidden)
        
    def forward(self,x , enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        
        # self-attention -> add&norm
        h = self.self_attention(x, dec_self_attn_mask)
        h = self.norm1(x + self.drop(self.proj1(h)))
        
        # encoder attention -> add&norm
        h2 = self.self_attention(enc_outputs, dec_enc_attn_mask, x_q=h)
        h = self.norm2(h + self.drop(self.proj2(h2))) 
        
        # feedforward network
        h = self.norm3(h + self.drop(self.pwff(h)))
        
        return h

class Transformer(nn.Module):
    """ Transformer with Self-Attentive Blocks"""
    def __init__(self, cfg):
        super().__init__()
        #====================encoder===========================
        self.encoder_embed = Embeddings(cfg, len(korean_vocab))
        self.encoder_blocks = nn.ModuleList([Encoder_Block(cfg) for _ in range(cfg.n_layers)])

        #====================decoder============================
        self.decoder_embed = Embeddings(cfg, len(english_vocab))
        self.decoder_blocks = nn.ModuleList([Decoder_Block(cfg) for _ in range(cfg.n_layers)])
        
        #=========================================================
        self.projection = nn.Linear(cfg.dim, len(english_vocab))
        
        
    def forward(self, enc_inputs, dec_inputs):
        #============encoder============
        h = self.encoder_embed(enc_inputs)
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        for block in self.encoder_blocks:
            h = block(h, enc_self_attn_mask)
            
        enc_outputs = h
        
        
        #============decoder============
        
        # self attention mask
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).float()
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs).float()
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

        # encoder attention mask
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)
        
        
        # embedding
        h = self.decoder_embed(dec_inputs)
        
        
        for block in self.decoder_blocks:
            h = block(h, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
        #============projection==========
        
        out = self.projection(h)
        
        return out
    
    def greedy_decoding(self, enc_inputs, start_token_index = 1, end_token_index = 2, generation_max_len=128):
        
        with torch.no_grad():
            batch_size, max_length = enc_inputs.size()
            generation_end_flag = [0 for i in range(batch_size)]
            predicted_sentences = []
            #=================encoding=============
            h = self.encoder_embed(enc_inputs)
            enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
            for block in self.encoder_blocks:
                h = block(h, enc_self_attn_mask)    
            enc_outputs = h

            #================ greedy decoding ==================
            # dec_inputs : (batch size, 1) 
            dec_inputs = torch.ones(batch_size, 1, device=enc_inputs.device) * start_token_index
            dec_inputs = dec_inputs.long()

            for i in range(generation_max_len):

                #====================== decoder =======================
                # self attention mask
                dec_self_attn_pad_mask = None
                dec_self_attn_subsequent_mask = None
                dec_self_attn_mask = None

                # encoder attention mask
                dec_enc_attn_mask = None


                # embedding
                h = self.decoder_embed(dec_inputs)


                for block in self.decoder_blocks:
                    h = block(h, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)


                out = self.projection(h[:,-1,:])
                pred = out.argmax(-1) 

#                 print(out.size(), pred)

                dec_inputs = torch.cat((dec_inputs, pred.unsqueeze(1)),dim=1)
    #             print(dec_inputs)

                predicted_sentences.append(pred)
                for j, boolean in enumerate(pred==end_token_index):
                    if boolean == True:
                        generation_end_flag[j] = 1
                if sum(generation_end_flag) == batch_size:
                    break

        return torch.stack(predicted_sentences, dim=1)
        
        

In [ ]:
# 한 -> 영
data_ = pd.read_csv("./korean_data/train.csv")
korean_data = data_["Korean"].values
english_data = data_["English"].values

In [ ]:
nltk.tokenize.word_tokenize(korean_data[100])

['그것을', '막기', '위해', ',', '많은', '과학자는', '친환경적인', '사용', '방법을', '연구했어요', '.']

In [ ]:
def build_dict(seqs):
    num_skip_sent = 0
    word_count = 4
    vocab = ["<pad>","<s>","</s>","<unk>"]
    word2id = {"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3}
    id2word = {0: "<pad>", 1: "<s>", 2: "</s>", 3: "<unk>"}
    print("Building vocab and dict..")
    for line in seqs:
        words = line.strip().split(' ') # tokenized by space 
        for word in words:
            if word not in vocab:
                word_count += 1 # increment word_count
                vocab.append(word) # append new unique word
                index = word_count - 1 # word index (consider index 0)
                word2id[word] = index # word to index
                id2word[index] = word # index to word
    
    print("Number of unique words: %d" % len(vocab))
    print("Finised building vocab and dict!")

    return vocab, word2id, id2word

In [ ]:
if os.path.isfile("./korean_vocab.pkl"):
    with open("./train_korean.pkl", "rb") as f:
        korean_data_token_ = pickle.load(f)
    with open("./train_english.pkl", "rb") as f:
        english_data_token_ = pickle.load(f)    
    
    with open("./korean_vocab.pkl", "rb") as f:
        korean_vocab = pickle.load(f)
    with open("./korean_word2id.pkl", "rb") as f:
        korean_word2id = pickle.load(f)
    with open("./korean_id2word.pkl", "rb") as f:
        korean_id2word = pickle.load(f)
    with open("./english_vocab.pkl", "rb") as f:
        english_vocab = pickle.load(f)
    with open("./english_word2id.pkl", "rb") as f:
        english_word2id = pickle.load(f)
    with open("./english_id2word.pkl", "rb") as f:
        english_id2word = pickle.load(f)

else:
    korean_data_token = []
    for sent in korean_data:
        korean_data_token.append([token for token in nltk.tokenize.word_tokenize(sent)])

    english_data_token = []
    for sent in english_data:
        english_data_token.append([token for token in nltk.tokenize.word_tokenize(sent)])

    korean_data_token_ = [' '.join(token) for token in korean_data_token]
    english_data_token_ = [' '.join(token) for token in english_data_token]    
    
    korean_vocab, korean_word2id, korean_id2word = build_dict(korean_data_token_)
    english_vocab, english_word2id, english_id2word = build_dict(english_data_token_)
    
    pickle.dump(korean_data_token_, open("./train_korean.pkl", "wb" ))
    pickle.dump(english_data_token_, open("./train_english.pkl", "wb" ))    
    
    pickle.dump(korean_vocab, open("./korean_vocab.pkl", "wb" ))
    pickle.dump(korean_word2id, open("./korean_word2id.pkl", "wb" ))
    pickle.dump(korean_id2word, open("./korean_id2word.pkl", "wb" ))

    pickle.dump(english_vocab, open("./english_vocab.pkl", "wb" ))
    pickle.dump(english_word2id, open("./english_word2id.pkl", "wb" ))
    pickle.dump(english_id2word, open("./english_id2word.pkl", "wb" ))

Building vocab and dict..


KeyboardInterrupt: ignored

In [ ]:
def batch(iterable, n=1):
    args = [iter(iterable)] * n
    return zip_longest(*args)


def pad_tensor(vec, pad, value=0, dim=0):
    """
    pad token으로 채우는 용도 
    args:
        vec - tensor to pad
        pad - the size to pad to
        dim - dimension to pad
    return:
        a new tensor padded to 'pad' in dimension 'dim'
    """
    pad_size = pad - vec.shape[0]

    if len(vec.shape) == 2:
        zeros = torch.ones((pad_size, vec.shape[-1])) * value
    elif len(vec.shape) == 1:
        zeros = torch.ones((pad_size,)) * value
    else:
        raise NotImplementedError
    return torch.cat([torch.Tensor(vec), zeros], dim=dim)


In [ ]:
def pad_collate(batch, values=(0, 0), dim=0):
    """
    데이터 로더에 들어가기전에 batch화 할 때 거치는 함수 
    args:
        batch - list of (tensor, label)
    reutrn:
        xs - a tensor of all examples in 'batch' after padding
        ys - a LongTensor of all labels in batch
        ws - a tensor of sequence lengths
    """

    sequence_lengths = torch.Tensor([int(x[0].shape[dim]) for x in batch]) # 각 batch 마다 길이를 얻어내고 
    sequence_lengths, xids = sequence_lengths.sort(descending=True) # 감소하는 순서로 정렬
    target_lengths = torch.Tensor([int(x[1].shape[dim]) for x in batch])
    # find longest sequence (가장 긴 sequence의 길이를 구함 )
    src_max_len = max(map(lambda x: x[0].shape[dim], batch))
    tgt_max_len = max(map(lambda x: x[1].shape[dim], batch))
    # pad according to max_len (max length 만큼 padd를 추가 )
    batch = [(pad_tensor(x, pad=src_max_len, dim=dim), pad_tensor(y, pad=tgt_max_len, dim=dim)) for (x, y) in batch]

    # stack all
    xs = torch.stack([x[0] for x in batch], dim=0)
    ys = torch.stack([x[1] for x in batch], dim=0)
    xs = xs[xids].contiguous() # decreasing order로 다시 나열 
    ys = ys[xids].contiguous() # xids 와 같은 순서로 
    target_lengths = target_lengths[xids] 
    return xs.long(), ys.long(), sequence_lengths.int(), target_lengths.int()


class ToyDataset(data.Dataset):
    """
    https://talbaumel.github.io/blog/attention/
    """
    def __init__(self,  ko_path, en_path , ko_word2id, en_word2id):
        with open(ko_path, "rb") as f:
            self.ko_seqs = pickle.load(f)
        with open(en_path, "rb") as f:
            self.en_seqs = pickle.load(f)
        self.ko_word2id = ko_word2id
        self.en_word2id = en_word2id


    def __len__(self):
        return len(self.ko_seqs)

    def __getitem__(self, index):
        ko_seqs = self.ko_seqs[index]
        en_seqs = self.en_seqs[index]
        ko_seqs = self.process(ko_seqs, self.ko_word2id)
        en_seqs = self.process(en_seqs, self.en_word2id)
        
        '''
        한국어 문장 -> 영어문장
        <s> 한국어 문장 인덱스들 </s>
        <s> 영어 문장 인덱스들 </s>
        '''
        
        return ko_seqs, en_seqs       

    def process(self, seq, word2id):
        '''
        <s> 한국어 문장 인덱스들 </s>
        <s> 영어 문장 인덱스들 </s>
        '''

        sequence = []
        sequence.append(word2id["<s>"])
        words = seq.strip().split(' ')
        for word in words:
            if len(sequence) < model_config.max_len:
                if word in word2id:
                    sequence.append(word2id[word]) # 
                else:
                    sequence.append(3) # replace by <unk> token
            else:
                break
        sequence.append(word2id["</s>"])
        sequence = torch.Tensor(sequence)
        return sequence

In [ ]:
sample_config = {
    "dim": 32,
    "dim_ff": 32,
    "n_layers": 2,
    "p_drop_attn": 0.1,
    "n_heads": 4,
    "p_drop_hidden": 0.1,
    "max_len": 30,
    "n_segments": 2,
    "vocab_size": 30522,
    "batch_size": 32
}

class AttributeDict(dict):
    def __getattr__(self, name):
        return self[name]

model_config = AttributeDict(sample_config)
model = Transformer(model_config)
model = model.cuda()
# out = model(sample[0].cuda(),sample[0].cuda())
# out.size()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
dataset = ToyDataset("train_korean.pkl", "train_english.pkl", korean_word2id, english_word2id)
train_loader = data.DataLoader(dataset, batch_size=model_config.batch_size, shuffle=True, collate_fn=pad_collate, drop_last=True)

In [ ]:

model.train()
for epoch in range(10):
    loss_list =[]
    for idx, batch in enumerate(train_loader):
        x, y, x_len, y_len = batch
        x = x.cuda()
        y = y.cuda()
        logits = model(x, y[:,:-1])
        loss = loss_fn(logits.view(-1,len(english_vocab)) , y[:,1:].contiguous().view(-1)) # loss 구하기 우리는 cross entropy 사용 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_list.append(loss.item())
        
        if (idx+1) % 400 == 0 :
            print('epoch {} iteration {}/{} loss {:.4f}'.format(epoch+1, idx+1, len(train_loader), np.mean(loss_list)))
            loss_list=[]
            
        if (idx+1) % 2000 == 0:
            with torch.no_grad():
                greedy_pred = model.greedy_decoding(x[0].unsqueeze(0))
            greedy_pred_indices = greedy_pred[0].data.cpu().tolist()
            pred_indices = logits[0].argmax(-1).data.cpu().tolist()
            input_indices = x[0].data.cpu().tolist()
            label_indices = y[0].data.cpu().tolist()
            
            if 2 in pred_indices:
                greedy_pred_len = pred_indices.index(2)
            else:
                greedy_pred_len = 128
            
            if 2 in pred_indices:
                pred_len = pred_indices.index(2)
            else:
                pred_len = 128

            if 2 in input_indices:
                input_len = input_indices.index(2)
            else:
                input_len = 128

            if 2 in label_indices:
                label_len = label_indices.index(2)
            else:
                label_len = 128
            
            greedy_pred_words = [english_id2word[idx] for i, idx in enumerate(greedy_pred_indices) if i<= greedy_pred_len]
            pred_words = [english_id2word[idx] for i, idx in enumerate(pred_indices) if i<=pred_len]
            input_words = [korean_id2word[idx] for i, idx in enumerate(input_indices) if i<=input_len]
            label_words = [english_id2word[idx] for i, idx in enumerate(label_indices) if i<=label_len]

            print('=====================================')
            print('입력:{}'.format(' '.join(input_words[1:-1])))
            print('출력(teacher forcing):{}'.format(' '.join(pred_words[:-1])))
            print('출력(greedy decoding):{}'.format(' '.join(greedy_pred_words[:-1])))
            print('정답:{}'.format(' '.join(label_words[1:-1])))
            print('=====================================')

In [ ]:
#결과값 출력
model.eval()
for idx, batch in enumerate(train_loader):
    x, y, x_len, y_len = batch
    x = x.cuda()
    y = y.cuda()
    pred = model.greedy_decoding(x)

    for i in range(pred.shape[0]):
        pred_indices = pred[i].data.cpu().tolist()
        input_indices = x[i].data.cpu().tolist()
        label_indices = y[i].data.cpu().tolist()

        if 2 in pred_indices:
            pred_len = pred_indices.index(2)
        else:
            pred_len = 128

        if 2 in input_indices:
            input_len = input_indices.index(2)
        else:
            input_len = 128
        
        if 2 in label_indices:
            label_len = label_indices.index(2)
        else:
            label_len = 128
        
        pred_words = [english_id2word[idx] for i,idx in enumerate(pred_indices) if i <= pred_len]
        input_words = [korean_id2word[idx] for i,idx in enumerate(input_indices) if i <= input_len]
        label_words = [english_id2word[idx] for i,idx in enumerate(label_indices) if i <= label_len]
        if pred_len>1:
            print('===================')
            print('입력:{}'.format(' '.join(input_words)))
            print('출력(greedy decoding):{}'.format(' '.join(pred_words)))
            print('답안:{}'.format(' '.join(label_words)))

# model save

In [ ]:
path = 'trained_model.pth'
torch.save(model.state_dict(), path)

## model load 

In [ ]:
path = 'trained_model.pth'
model.load_state_dict(torch.load(path))

## TEST

In [ ]:
def test_pad_collate(batch, values=(0, 0), dim=0):
    """
    데이터 로더에 들어가기전에 batch화 할 때 거치는 함수 
    args:
        batch - list of (tensor, label)
    reutrn:
        xs - a tensor of all examples in 'batch' after padding
        ys - a LongTensor of all labels in batch
        ws - a tensor of sequence lengths
    """

    sequence_lengths = torch.Tensor([int(x.shape[dim]) for x in batch]) # 각 batch 마다 길이를 얻어내고 
    # find longest sequence (가장 긴 sequence의 길이를 구함 )
    src_max_len = max(map(lambda x: x.shape[dim], batch))
    # pad according to max_len (max length 만큼 padd를 추가 )
    batch = [pad_tensor(x, pad=src_max_len, dim=dim) for x in batch]

    # stack all
    xs = torch.stack(batch, dim=0)
    return xs.long(), sequence_lengths.int()

class TestDataset(data.Dataset):
    def __init__(self,  ko_path, ko_word2id):
        self.load_data(ko_path)
        self.ko_word2id = ko_word2id
        
        
    def load_data(self, ko_path):
        korean_test_data_ = pd.read_csv("./korean_data/test_korean.csv")
        korean_test_data =korean_test_data_['Korean'].iloc[:2000].values
        korean_test_data_token = []
        for sent in korean_test_data:
            korean_test_data_token.append([token for token in nltk.tokenize.word_tokenize(sent)])
        korean_test_data_token = [' '.join(token) for token in korean_test_data_token]
        self.ko_seqs = korean_test_data_token
        
    def __len__(self):
        return len(self.ko_seqs)

    def __getitem__(self, index):
        ko_seqs = self.ko_seqs[index]
        ko_seqs = self.process(ko_seqs, self.ko_word2id)
        return ko_seqs       

    def process(self, seq, word2id):
        sequence = []
        sequence.append(word2id["<s>"])
        words = seq.strip().split(' ')
        for word in words:
            if word in word2id:
                sequence.append(word2id[word]) # 
            else:
                sequence.append(3) # replace by <unk> token
        sequence.append(word2id["</s>"])
        sequence = torch.Tensor(sequence)
        return sequence
    
    
def test(model):
    
    with open("./korean_word2id.pkl", "rb") as f:
        korean_word2id = pickle.load(f)
    
    test_dataset = TestDataset('./test_korean.pkl', korean_word2id)
    test_loader = data.DataLoader(test_dataset, batch_size=model_config.batch_size, shuffle=False, collate_fn=test_pad_collate, drop_last=False)
    model.eval()
    j = 0
    
    f = open('prediction_result.txt', 'w')
    for idx, batch in enumerate(test_loader):
        x, x_len= batch
        x = x.cuda()
        pred = model.greedy_decoding(x)

        for i in range(pred.shape[0]):
            j+=1
            
            pred_indices = pred[i].data.cpu().tolist()
            input_indices = x[i].data.cpu().tolist()

            if 2 in pred_indices:
                pred_len = pred_indices.index(2)
            else:
                pred_len = 128

            if 2 in input_indices:
                input_len = input_indices.index(2)
            else:
                input_len = 128

            pred_words = [english_id2word[idx] for i,idx in enumerate(pred_indices) if i <= pred_len]
            input_words = [korean_id2word[idx] for i,idx in enumerate(input_indices) if i <= input_len]
            
            if j%50 == 0 :
                print('========== index {} ========='.format(j))
                print('입력:{}'.format(' '.join(input_words[1:-1])))
                print('출력:{}'.format(' '.join(pred_words[:-1])))

            f.write(' '.join(pred_words[:-1]))
            f.write('\n')
            

In [ ]:
test(model)


# 과제

1. query, key, value 간의 관계에 대해 작성해주세요.
- 답변 : 기계번역이라면, 다음에 올 토큰이 무엇인지 묻는 것이 query, 정답에 대한 여러 후보들이 key, 후보들의 값을 value가 됩니다.
2. multi-head의 필요성에 대해 작성해주세요.
- 답변 : 여러 head로 self-attention을 시켜 단어의 여러 의미를 파악할 수 있도록 도와줍니다.
3. BERT 사전 학습의 두가지 태스크에 대해 설명해주세요.
- 답변 : MLM(Masked Language Model)과 NSP(Next Sentence Prediction)입니다.   

4. Transformer embedding과 BERT embedding의 차이점 두가지를 작성해주세요.
- 답변 : Transformer embedding에서는 Positional Encoding을 사용하고, BERT embedding에서는 대신에 Positional Embeddings와 Segment Embedding을 사용합니다. Segment Embedding은 단어가 첫번째 문장에 속하는지 두번째 문장에 속하는지 알려주는 역할을 하고, Positional Embedding은 각 단어가 첫번째인지, 두번째인지를 의미하는 Embedding값을 의미합니다.